In [1]:
import os
import glob
import pandas as pd
from argparse import ArgumentParser
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import GroupKFold

In [25]:
from signet.configs.Conv1D_LSTM_CTC_Loss import Conv1D_LSTM_CTC_Loss

In [26]:
configs = Conv1D_LSTM_CTC_Loss()

In [35]:
len(sorted(configs.POINT_LANDMARKS))

118

In [19]:
import plotly.express as px

In [2]:
ALL_FILENAMES = glob.glob( '../dataset/npy_data/*.npy')
train_df = pd.read_csv("../dataset/train.csv")
supp_df = pd.read_csv("../dataset/supplemental_metadata.csv")

print(len(ALL_FILENAMES))
usable_names = set()
for n in ALL_FILENAMES:
    usable_names.add(os.path.split(n)[-1])
gkf = GroupKFold(n_splits=5)

120165


In [3]:
seq_length = {}
for fpth in tqdm(ALL_FILENAMES):
    seq_length[os.path.split(fpth)[-1].split(".")[0]] = len(np.load(fpth))

100%|██████████| 120165/120165 [01:11<00:00, 1687.38it/s]


In [4]:
train_df["seqlen"] = train_df.sequence_id.apply(lambda x: seq_length[str(x)])
train_df["labellen"] = train_df.phrase.apply(len)

In [5]:
(train_df.seqlen>train_df.labellen).mean()

0.9651827163432924

In [6]:
supp_df["seqlen"] = supp_df.sequence_id.apply(lambda x: seq_length[str(x)] if str(x) in seq_length else 0)
supp_df["labellen"] = supp_df.phrase.apply(len)

In [7]:
(supp_df.seqlen>supp_df.labellen).mean()

0.965387665697345

In [8]:
supp_df = supp_df[supp_df.seqlen>supp_df.labellen].reset_index(drop=True)
train_df = train_df[train_df.seqlen>train_df.labellen].reset_index(drop=True)

In [9]:
len(train_df),len(supp_df)

(64868, 51125)

In [11]:
all_df = pd.concat([train_df,supp_df])

In [24]:
(all_df.seqlen>384).mean(),(all_df.labellen>45).mean()

(0.013173208728112904, 0.0)

In [22]:
px.histogram(all_df.sample(1000),x=["seqlen","labellen"])

In [11]:
a23+1
for fold_idx,(train, test) in enumerate(gkf.split(train_df.sequence_id, groups=train_df.participant_id)):
    valid_seqid = set()
    for idx in test:
        valid_seqid.add(train_df.sequence_id[idx])
    train_info = []
    valid_info = []
    for seqid,label in zip(train_df.sequence_id,train_df.phrase):
        fname = f"{seqid}.npy"
        if fname in usable_names and seqid not in valid_seqid:
            train_info.append((fname,label))
    for seqid,label in zip(supp_df.sequence_id,supp_df.phrase):
        fname = f"{seqid}.npy"
        if fname in usable_names:
            train_info.append((fname,label))
    train_info = pd.DataFrame(train_info,columns=["files","labels"])

    for seqid,label in zip(train_df.sequence_id,train_df.phrase):
        fname = f"{seqid}.npy"
        if fname in usable_names and seqid in valid_seqid:
            valid_info.append((fname,label))
    valid_info = pd.DataFrame(valid_info,columns=["files","labels"])
    print(len(valid_info),len(train_info))
    train_info.to_csv(f"../dataset/folds/fold{fold_idx}_train.csv",index=False)
    valid_info.to_csv(f"../dataset/folds/fold{fold_idx}_valid.csv",index=False)

13053 102940
12929 103064
12909 103084
13004 102989
12973 103020
